# 🎓 FrugalGPT: Performance and Cost Tradeoffs

This notebook illustrates the FrugalGPT framework for _building LLM Applications with budget constraints._

In particular, we will focus on evaluating the performance and cost tradeoffs enabled by FrugalGPT.

NB: You are highly suggested to use accelerated hardware (GPU/TPU) to run this notebook.

## Installation
Let us start by installing FrugalGPT (if you haven't yet!).

In [1]:
# set up the environment
%%capture
! git clone https://github.com/stanford-futuredata/FrugalGPT
! pip install git+https://github.com/stanford-futuredata/FrugalGPT
!mkdir -p FrugalGPT/strategy/
! wget  https://github.com/lchen001/DataHolder/releases/download/v0.0.3/HEADLINES_Model20241128.zip
! unzip HEADLINES_Model20241128.zip -d FrugalGPT/strategy/HEADLINES_Model20241128
! rm HEADLINES_Model20241128.zip
! wget -P FrugalGPT/db/ https://github.com/lchen001/DataHolder/releases/download/v0.0.3/HEADLINES.sqlite


In [2]:
%cd FrugalGPT

/content/FrugalGPT


In [3]:
%load_ext autoreload
%autoreload 2
import sys, json, copy
import logging
logging.disable(logging.CRITICAL)
sys.path.append("src/")

## Setup
Next, let us set up the environment and API keys. You do _not_ need API keys to run the notebook! They are only needed if you want to use FrugalGPT for your own queries.
#### NB: _For your own queries, not all API keys are needed, too. If you only want to leverage LLMs from, e.g., OpenAI and AI21, setting up API keys for them is sufficient._

In [4]:
import os
os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_KEY'
os.environ['AI21_STUDIO_API_KEY'] = 'YOUR_AI21_KEY'
os.environ['COHERE_STUDIO_API_KEY'] = 'YOUR_COHERE_KEY'
os.environ['TEXTSYNTH_API_SECRET_KEY'] = 'YOUR_TEXTSYNTH_KEY'
os.environ['ANTHROPIC_API_KEY'] = 'YOUR_ANTHROPIC_KEY'
os.environ['TOGETHER_API_KEY'] = 'YOUR_TOGETHER_KEY'
os.environ['GEMINI_API_KEY'] = 'YOUR_GEMINI_KEY'

from IPython.display import display
import FrugalGPT
supported_LLM = FrugalGPT.getservicename()
print("supported LLMs:",supported_LLM)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

supported LLMs: ['textsynth/gptneox_20B', 'textsynth/fairseq_gpt_13B', 'textsynth/gptj_6B', 'openai/text-davinci-002', 'openai/text-davinci-003', 'openai/text-curie-001', 'openai/text-babbage-001', 'openai/text-ada-001', 'openaichat/gpt-4o-mini', 'openaichat/gpt-4o-mini-2024-07-18', 'openaichat/gpt-4o', 'openaichat/gpt-4o-2024-05-13', 'openaichat/gpt-4-turbo', 'openaichat/gpt-4o-2024-08-06', 'openaichat/gpt-3.5-turbo', 'openaichat/gpt-4', 'ai21/jamba-1.5-mini', 'ai21/jamba-1.5-large', 'ai21/j1-jumbo', 'ai21/j1-grande', 'ai21/j1-large', 'ai21/j2-ultra', 'ai21/j2-mid', 'ai21/j2-light', 'cohere/command', 'cohere/base', 'cohere/xlarge', 'cohere/medium', 'togetherai/Qwen/Qwen2-72B-Instruct', 'togetherai/mistralai/Mistral-7B-Instruct-v0.1', 'togetherai/google/gemma-2b-it', 'togetherai/google/gemma-2-9b-it', 'togetherai/google/gemma-2-27b-it', 'togetherai/meta-llama/Meta-Llama-3-8B-Instruct-Lite', 'togetherai/Qwen/Qwen1.5-110B-Chat', 'togetherai/mistralai/Mistral-7B-Instruct-v0.3', 'togethera

## Generating the tradeoffs involves three major steps: (i) prepare the dataset, (ii) train the FrugalGPT strategy, and (iii) evaluate and save the performance.

## Step 1: Prepare the dataset

In [5]:
dataname = "HEADLINES"

test_data = FrugalGPT.loadcsvdata(f"data/{dataname}/test.csv")
prefix = open(f'config/prompt/{dataname}/prefix_e8.txt').read()
test_data = FrugalGPT.formatdata(test_data,prefix)

train_data = FrugalGPT.loadcsvdata(f"data/{dataname}/train.csv")
prefix = open(f'config/prompt/{dataname}/prefix_e8.txt').read()
train_data = FrugalGPT.formatdata(train_data,prefix)

## Step 2: Train the FrugalGPT strategy for different budgets

Let us first evaluate individual models.

In [6]:
import pandas as pd

def generate_dataframe(service_names, train_data, test_data, genparams,db_path="db/SCIQ.sqlite",
                       max_workers=2):
    # Initialize an empty list to store the rows for the DataFrame
    data = []
    MyLLMforAll = FrugalGPT.LLMforAll(
                     db_path=db_path,
                     max_workers=max_workers,

)
    # Dictionary to keep track of markers for each provider
    provider_marker = {}

    # Iterate through the service names
    for name in service_names:
        # Extract provider and method
        provider = name.split('/')[0]
        method = name.split('/')[-1]

        # If the provider is seen for the first time, initialize its marker
        if provider not in provider_marker:
            provider_marker[provider] = 1
        else:
            provider_marker[provider] += 1
        # Get the completion batch for train and test data
        r1_train = MyLLMforAll.get_completion_batch(queries=train_data, genparams=genparams, service_name=name)
        r2_train = FrugalGPT.compute_score(r1_train)
        r1_test = MyLLMforAll.get_completion_batch(queries=test_data, genparams=genparams, service_name=name)
        r2_test = FrugalGPT.compute_score(r1_test)

        # Extract accuracy and cost
        train_acc = r2_train['em']
        train_cost = r2_train['cost']
        test_acc = r2_test['em']
        test_cost = r2_test['cost']

        # Create a row with the schema
        row = {
            "Test_acc": test_acc,
            "Test_cost": test_cost,
            "Test_size": len(test_data),
            "Train_acc": train_acc,
            "Train_cost": train_cost,
            "Train_size": len(train_data),
            "Budget": 10,
            "Method": method,
            "Provider": provider,
            "Marker": provider_marker[provider],
        }

        # Append the row to the data list
        data.append(row)

    # Create the DataFrame from the data list
    df = pd.DataFrame(data)

    return df

In [7]:

service_names = [
    'ai21/jamba-1.5-large',

    'togetherai/meta-llama/Meta-Llama-3-70B-Instruct-Turbo',
    'togetherai/google/gemma-2-9b-it',

    'google/gemini-1.5-flash',
    'google/gemini-1.5-pro',
    'google/gemini-1.5-flash-8b',

    'openaichat/gpt-4o-2024-05-13',
    'openaichat/gpt-4o-mini',
    'openaichat/gpt-4-turbo',

    'anthropic/claude-3-5-sonnet-20240620',
                 ]

genparams=FrugalGPT.GenerationParameter(max_tokens=50, temperature=0.1, stop=['\n'])

sample_size = 10000
individualmodel_df = generate_dataframe(service_names,
                                        train_data[0:sample_size], test_data[0:sample_size],
                                        genparams,
                                        db_path=f"db/{dataname}.sqlite",
                                        max_workers=2)
display(individualmodel_df)
individualmodel_df.to_csv(f"summary_{dataname}_e8_2024.csv")

5000it [00:07, 648.90it/s]
5000it [00:07, 663.09it/s]
5000it [00:06, 744.89it/s]
5000it [00:06, 724.65it/s]
5000it [00:06, 766.44it/s]
5000it [00:06, 732.43it/s]
5000it [00:07, 691.19it/s]
5000it [00:06, 718.24it/s]
5000it [00:07, 706.94it/s]
5000it [00:07, 701.42it/s]
5000it [00:07, 705.81it/s]
5000it [00:07, 685.83it/s]
5000it [00:06, 746.64it/s]
5000it [00:06, 734.58it/s]
5000it [00:07, 710.12it/s]
5000it [00:06, 756.32it/s]
5000it [00:06, 735.04it/s]
5000it [00:06, 748.32it/s]
5000it [00:07, 672.67it/s]
5000it [00:07, 680.63it/s]


,Test_acc,Test_cost,Test_size,Train_acc,Train_cost,Train_size,Budget,Method,Provider,Marker
0,0.8142,0.000519,5000,0.8146,0.000519,5000,10,jamba-1.5-large,ai21,1
1,0.8264,0.000201,5000,0.8290,0.000201,5000,10,Meta-Llama-3-70B-Instruct-Turbo,togetherai,1
2,0.8320,0.000070,5000,0.8336,0.000070,5000,10,gemma-2-9b-it,togetherai,2
3,0.8480,0.000017,5000,0.8442,0.000017,5000,10,gemini-1.5-flash,google,1
4,0.8250,0.000290,5000,0.8234,0.000290,5000,10,gemini-1.5-pro,google,2
5,0.8422,0.000009,5000,0.8462,0.000009,5000,10,gemini-1.5-flash-8b,google,3
6,0.8332,0.001100,5000,0.8318,0.001100,5000,10,gpt-4o-2024-05-13,openaichat,1
7,0.8478,0.000033,5000,0.8506,0.000033,5000,10,gpt-4o-mini,openaichat,2
8,0.8558,0.003331,5000,0.8616,0.003331,5000,10,gpt-4-turbo,openaichat,3
9,0.8422,0.000807,5000,0.8432,0.000807,5000,10,claude-3-5-sonnet-20240620,anthropic,1


In [8]:
from google.colab import files
#files.download(f'db/HEADLINES.sqlite')

Now let us train FrugalGPT on this dataset.

In [9]:
import numpy
from tqdm import tqdm
def compute_tradeoffs(
    train_data,
                      budget_list,
                      name = "test",

                      service_names = ['openaichat/gpt-4o-mini',
                                       'openaichat/gpt-4o',
                                      'openaichat/gpt-4-turbo',
                 'togetherai/meta-llama/Meta-Llama-3-70B-Instruct-Turbo',
                                      'togetherai/google/gemma-2-9b-it',
                 ],
                      prefix="",
                      skip=0,
    MyCascade = FrugalGPT.LLMCascade(
          score_noise_injection=False,
  db_path="db/SCIQ.sqlite",
  ),

    cascade_depth=3,
    score_test_size=0.55,

                      ):

  for idx,budget in tqdm(enumerate(budget_list)):
    # train the model
    user_budget = budget
    #MyCascade.load(loadpath=f"strategy/{name}/",budget=user_budget)

    try:
      MyCascade.load(loadpath=f"strategy/{name}/",budget=user_budget)
      print("Already trained. Skipped.")
      continue
    except:
      print("cannot find, start new training")
    if(idx<skip):
      continue
    if(idx==0):
        result = MyCascade.train(train_data,budget=user_budget,
                                 service_names=service_names,
                                 no_scorer_train=False,
                                 prefix=prefix,
                                 cascade_depth=cascade_depth,
                                 score_test_size=score_test_size,
                                 )
    else:
      result = MyCascade.train(train_data,budget=user_budget,
                               service_names=service_names,
                               no_scorer_train=True,
                               prefix=prefix,
                               cascade_depth=cascade_depth,
                               score_test_size=score_test_size,
                               )
    MyCascade.save(savepath=f"strategy/{name}/")
  return

In [10]:
start_budget = 0.0000171
end_budget = 0.004
num_eval = 20

name = f'{dataname}_Model20241128'
budget_list = numpy.linspace(start_budget, end_budget, num_eval)
budget_list[0] = 0.0000105
# load data
dev = FrugalGPT.loadcsvdata(f"data/{dataname}/train.csv")
train_data = FrugalGPT.formatdata(dev,prefix)
MyCascade= FrugalGPT.LLMCascade(
          score_noise_injection=False,
  db_path=f"db/{dataname}.sqlite",
  batch_build=True,
  )
#MyCascade.load(loadpath=f"strategy/{name}/",budget=0.00017)

In [11]:
#'''
service_names_train = [
    'togetherai/meta-llama/Meta-Llama-3-70B-Instruct-Turbo',

    'google/gemini-1.5-flash',
    'google/gemini-1.5-flash-8b',

    #'openaichat/gpt-4o-mini',
    'openaichat/gpt-4-turbo',

    'anthropic/claude-3-5-sonnet-20240620',
                 ]
print(name)
compute_tradeoffs(train_data=train_data,
                  budget_list=budget_list,
                  name=name,
                  service_names=service_names_train,
                  prefix=prefix,
                  skip=0, # you can manually skip the first few budgets if they have already been trained.
                  MyCascade=MyCascade,
                  cascade_depth=3,
                  score_test_size=0.45,
                  )
#'''

HEADLINES_Model20241128


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Already trained. Skipped.


19it [00:22,  1.11s/it]

Already trained. Skipped.


20it [00:23,  1.18s/it]

Already trained. Skipped.


In [12]:
'''
# Test
MyCascade2 = FrugalGPT.LLMCascade(
          score_noise_injection=False,
  db_path=f"db/{dataname}.sqlite",
  batch_build=True,
  )

#print(name)
compute_tradeoffs(train_data=train_data,
                  budget_list=budget_list,
                  #name=name,
                  #service_names=service_names,
                  name = 'testtest',
                  service_names=['openaichat/gpt-4o-mini',
                                'openaichat/gpt-4-turbo',
                                'togetherai/google/gemma-2-9b-it',
                 ],
                  prefix=prefix,
                  skip=0, # you can manually skip the first few budgets if they have already been trained.
                  MyCascade=MyCascade2,
                  cascade_depth=2,
                  )
'''

'\n# Test\nMyCascade2 = FrugalGPT.LLMCascade(\n          score_noise_injection=False,\n  db_path=f"db/{dataname}.sqlite",\n  batch_build=True,\n  )\n\n#print(name)\ncompute_tradeoffs(train_data=train_data,\n                  budget_list=budget_list,\n                  #name=name,\n                  #service_names=service_names,\n                  name = \'testtest\',\n                  service_names=[\'openaichat/gpt-4o-mini\',\n                                \'openaichat/gpt-4-turbo\',\n                                \'togetherai/google/gemma-2-9b-it\',\n                 ],\n                  prefix=prefix,\n                  skip=0, # you can manually skip the first few budgets if they have already been trained.\n                  MyCascade=MyCascade2,\n                  cascade_depth=2,\n                  )\n'

In [13]:
#'''
import shutil
folder_to_zip = f'strategy/{name}'
output_zip_file = f'{name}.zip'
shutil.make_archive(output_zip_file.replace('.zip', ''), 'zip', folder_to_zip)
print(f"Folder '{folder_to_zip}' zipped as '{output_zip_file}'.")
files.download(output_zip_file)
#'''

Folder 'strategy/HEADLINES_Model20241128' zipped as 'HEADLINES_Model20241128.zip'.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Step 3: Evaluate and save the performance

In [14]:
def generate_dataframe_from_cascade(MyCascade,budget_list, train_data, test_data, genparams,name):
    # Initialize an empty list to store the rows for the DataFrame
    data = []

    # Iterate through the budget list
    for budget in tqdm(budget_list):
        # Load the strategy for the given budget
        MyCascade.load(loadpath=f"strategy/{name}/", budget=budget)

        # Get the completion batch for train data
        train_result = MyCascade.get_completion_batch(queries=train_data, genparams=genparams)

        # Compute the ACC and cost for train data
        train_acc_cost = FrugalGPT.compute_score(train_result)


        # Get the completion batch for test data
        test_result = MyCascade.get_completion_batch(queries=test_data, genparams=genparams)

        # Compute the ACC and cost for test data
        test_acc_cost = FrugalGPT.compute_score(test_result)

        # Create a row with the schema
        row = {
            "Test_acc": test_acc_cost['em'],
            "Test_cost": test_acc_cost['cost'],
            "Test_size": len(test_data),
            "Train_acc": train_acc_cost['em'],
            "Train_cost": train_acc_cost['cost'],
            "Train_size": len(train_data),
            "Budget": budget,
            "Method": "FrugalGPT",
            "Provider": "FrugalGPT",
            "Marker": 1,  # Marker is always 1 for this function
        }

        # Append the row to the data list
        data.append(row)
        display(row)

    # Create the DataFrame from the data list
    df = pd.DataFrame(data)

    return df

In [15]:
MyCascade_eval = FrugalGPT.LLMCascade()
MyCascade_eval.prefix = prefix
frugalgpt_df = generate_dataframe_from_cascade(MyCascade_eval,
                                               budget_list, train_data, test_data, genparams,
                                               name)
display(frugalgpt_df)
frugalgpt_df.to_csv(f"summary_{dataname}_e8_frugalgpt_2024.csv")

  0%|          | 0/20 [00:00<?, ?it/s]

{'Test_acc': 0.8522,
 'Test_cost': 9.787287999999999e-06,
 'Test_size': 5000,
 'Train_acc': 0.858,
 'Train_cost': 1.0138984e-05,
 'Train_size': 5000,
 'Budget': 1.05e-05,
 'Method': 'FrugalGPT',
 'Provider': 'FrugalGPT',
 'Marker': 1}

  5%|▌         | 1/20 [01:23<26:23, 83.32s/it]

{'Test_acc': 0.8698,
 'Test_cost': 0.00019212226300000003,
 'Test_size': 5000,
 'Train_acc': 0.8802,
 'Train_cost': 0.00022440889700000004,
 'Train_size': 5000,
 'Budget': 0.0002267263157894737,
 'Method': 'FrugalGPT',
 'Provider': 'FrugalGPT',
 'Marker': 1}

 10%|█         | 2/20 [02:52<26:02, 86.78s/it]

...

{'Test_acc': 0.8746,
 'Test_cost': 0.0013588096,
 'Test_size': 5000,
 'Train_acc': 0.8894,
 'Train_cost': 0.00139017124,
 'Train_size': 5000,
 'Budget': 0.0037903736842105264,
 'Method': 'FrugalGPT',
 'Provider': 'FrugalGPT',
 'Marker': 1}

 95%|█████████▌| 19/20 [37:45<02:06, 126.84s/it]

{'Test_acc': 0.8746,
 'Test_cost': 0.0013588096,
 'Test_size': 5000,
 'Train_acc': 0.8894,
 'Train_cost': 0.00139017124,
 'Train_size': 5000,
 'Budget': 0.004,
 'Method': 'FrugalGPT',
 'Provider': 'FrugalGPT',
 'Marker': 1}

100%|██████████| 20/20 [39:49<00:00, 119.50s/it]


,Test_acc,Test_cost,Test_size,Train_acc,Train_cost,Train_size,Budget,Method,Provider,Marker
0,0.8522,0.000010,5000,0.8580,0.000010,5000,0.000010,FrugalGPT,FrugalGPT,1
1,0.8698,0.000192,5000,0.8802,0.000224,5000,0.000227,FrugalGPT,FrugalGPT,1
2,0.8744,0.000384,5000,0.8866,0.000428,5000,0.000436,FrugalGPT,FrugalGPT,1
3,0.8740,0.000553,5000,0.8882,0.000582,5000,0.000646,FrugalGPT,FrugalGPT,1
4,0.8740,0.000553,5000,0.8882,0.000582,5000,0.000856,FrugalGPT,FrugalGPT,1
5,0.8732,0.000998,5000,0.8884,0.001015,5000,0.001065,FrugalGPT,FrugalGPT,1
6,0.8732,0.000998,5000,0.8884,0.001015,5000,0.001275,FrugalGPT,FrugalGPT,1
7,0.8746,0.001359,5000,0.8894,0.001390,5000,0.001484,FrugalGPT,FrugalGPT,1
8,0.8746,0.001359,5000,0.8894,0.001390,5000,0.001694,FrugalGPT,FrugalGPT,1
9,0.8746,0.001359,5000,0.8894,0.001390,5000,0.001904,FrugalGPT,FrugalGPT,1


Now let us save the results to local disk.

In [16]:
from google.colab import files
import copy
individualmodel_df2 = copy.copy(individualmodel_df)
#individualmodel_df2['Test_cost'] = individualmodel_df2['Test_cost'] * individualmodel_df2['Test_size']
full_pd = pd.concat([frugalgpt_df,individualmodel_df2,])
full_pd.to_csv(f"summary_{dataname}_e8_full_2024.csv")
files.download(f'summary_{dataname}_e8_full_2024.csv')
display(full_pd)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Test_acc,Test_cost,Test_size,Train_acc,Train_cost,Train_size,Budget,Method,Provider,Marker
0,0.8522,0.000010,5000,0.8580,0.000010,5000,0.000010,FrugalGPT,FrugalGPT,1
1,0.8698,0.000192,5000,0.8802,0.000224,5000,0.000227,FrugalGPT,FrugalGPT,1
2,0.8744,0.000384,5000,0.8866,0.000428,5000,0.000436,FrugalGPT,FrugalGPT,1
3,0.8740,0.000553,5000,0.8882,0.000582,5000,0.000646,FrugalGPT,FrugalGPT,1
4,0.8740,0.000553,5000,0.8882,0.000582,5000,0.000856,FrugalGPT,FrugalGPT,1
5,0.8732,0.000998,5000,0.8884,0.001015,5000,0.001065,FrugalGPT,FrugalGPT,1
6,0.8732,0.000998,5000,0.8884,0.001015,5000,0.001275,FrugalGPT,FrugalGPT,1
7,0.8746,0.001359,5000,0.8894,0.001390,5000,0.001484,FrugalGPT,FrugalGPT,1
8,0.8746,0.001359,5000,0.8894,0.001390,5000,0.001694,FrugalGPT,FrugalGPT,1
9,0.8746,0.001359,5000,0.8894,0.001390,5000,0.001904,FrugalGPT,FrugalGPT,1


In [17]:
'''
import time
i=0
while(1):
  print(i)
  i+=1
  time.sleep(60)
'''

'\nimport time\ni=0\nwhile(1):\n  print(i)\n  i+=1\n  time.sleep(60)\n'